In [1]:
import numpy as np
import nilearn
import os
import glob

main_folder = 'C:\\Users\\davide\\Documents\\GitHub\\wp1_2_roast\\' 
output_folder = 'D:\\roast-chapter3\\wp_all_results\\' # where to save results
data_folder = 'D:\\roast-chapter3\\'

# Datasets names and subjects lists
db_names = ['wp2a']

# list of subjects with cod-liver oil in T1
db_subjects = [['01','02','03','04','06','07','08','09','10','11','12','13','15','16','17','18','19','20','22','23','24']] 

# List of coordinates for each subject (return electrode: cathodal, active electrode: anodal)
cath_coords = {
            '01': (140,188,185),
            '02': (133,192,182),
            '03': (127,207,195),
            '04': (139,195,180),
            '06': (135,186,188),
            '07': (142,183,196),
            '08': (148,182,188),
            '09': (142,186,187),
            '10': (151,180,182),
            '11': (149,189,179),
            '12': (146,185,189),
            '13': (145,191,189),
            '14': (131,186,202),
            '15': (134,192,193),
            '16': (133,185,193),
            '17': (133,185,190),
            '18': (140,190,185),
            '19': (158,183,172),
            '20': (156,174,190),
            '22': (147,179,181),
            '23': (142,187,180),
            '24': (132,182,190)
}
anod_coords = {
            '01': (70,85,221),
            '02': (49,99,211),
            '03': (37,98,200),
            '04': (38,109,197),
            '06': (36,86,189),
            '07': (46,75,213),
            '08': (40,75,193),
            '09': (37,88,194),
            '10': (40,75,190),
            '11': (49,81,211),
            '12': (49,66,203),
            '13': (49,94,216),
            '14': (46,60,209),
            '15': (36,73,199),
            '16': (47,76,204),
            '17': (40,83,204),
            '18': (46,99,216),
            '19': (28,97,190),
            '20': (40,72,198),
            '22': (46,94,210),
            '23': (44,93,207),
            '24': (37,95,199)
}

# list of subjects with cod-liver oil in T2
db_subjects_t2s = [['03','04','06','07','08','09','10','11','12','13','14','17','18','19','20','22','23','24']]


In [2]:
from nilearn import image
from skimage import measure
from nilearn.image import new_img_like
from nilearn.plotting import plot_stat_map, plot_anat, plot_img
import cupy as cp
from scipy import ndimage
from tqdm import tqdm

for db_id, db in enumerate(db_names): # Iterate all three datasets
    db_path = os.path.join(data_folder, db) # Dataset folder

    for i, subject in tqdm(enumerate(db_subjects[db_id])): #Iterate all subjects
        path = db_path + '\sub-' + subject # Subject folder
        print(path)

        T1 = image.load_img(glob.glob(path + '\sub-*T1.nii'))

        # filename of the t1 without the cod liver oil
        T1_cleaned_fname = glob.glob(path + '\sub-*T1.nii')[0].split('\\')[-1].split('.')[0] + '_withoutcdl.nii'
        t1_cdl = image.load_img(path + '\\' + 'cod_liver_oilT1.nii')
        T1_itk_snap = image.load_img(path + '\\' + T1_cleaned_fname)
        t1_itk_mask = image.load_img(glob.glob(path + '\\t1_elect_mask.nii'))



C:\Users\davide\miniconda3\envs\neuroimg\lib\site-packages\nilearn\datasets\__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "
1it [00:00,  8.22it/s]

D:\roast-chapter3\wp2a\sub-07
